In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import GridSearchCV
from skimage.feature import hog
from skimage.io import imread
from skimage.transform import rescale

with open('image96.txt','r') as f:
    filename = f.readlines()

filename = [_.split('\n')[0] for _ in filename]
label = pd.read_csv('label.csv',encoding='CP949')
label = label[['강수15','fog','weather','sunny']]

weatherlabel = {'맑음' : 0, '안개':1,'비': 2}
foglabel = {'None' : 0, 'Shallow':1,'Dense':2}
label['weather'] = label['weather'].map(weatherlabel)
label['fog'] = label['fog'].map(foglabel)

C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Gyu\Anaconda3\envs\py37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
label1 = label.iloc[:80000]
filename1 = filename[:80000]

In [3]:

X_train, X_test, y_train, y_test = train_test_split(filename1, label1, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [4]:
flat_data = []
for i, direc in enumerate(X_train):
    img = imread(direc,as_gray=True)
#     img_resized = resize(img, (60, 60), anti_aliasing=True, mode='reflect')
    flat_data.append(img)

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
   
class HogTransformer(BaseEstimator, TransformerMixin):
    """
    Expects an array of 2d arrays (1 channel images)
    Calculates hog features for each img
    """
 
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
 
        def local_hog(X):
            return hog(X,
                       orientations=self.orientations,
                       pixels_per_cell=self.pixels_per_cell,
                       cells_per_block=self.cells_per_block,
                       block_norm=self.block_norm)
 
        try: # parallel
            return np.array([local_hog(img) for img in X])
        except:
            return np.array([local_hog(img) for img in X])

In [6]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
import skimage
 
hogify = HogTransformer(
    pixels_per_cell=(8, 8),
    cells_per_block=(2,2),
    orientations=9,
    block_norm='L2-Hys'
)
# scalify = StandardScaler()
X_train_hog = hogify.fit_transform(flat_data)
# X_train_prepared = scalify.fit_transform(X_train_hog)

In [33]:
hogify1 = HogTransformer(
    pixels_per_cell=(10, 10),
    cells_per_block=(2,2),
    orientations=9,
    block_norm='L2-Hys'
)

hog1 = hogify1.fit_transform(flat_data[:3])
hog1.shape

(3, 2304)

In [19]:
scalify = StandardScaler()
X_train_prepared = scalify.fit_transform(X_train_hog)

In [20]:
sgd_clf = SGDClassifier(random_state=42, tol=1e-3)
sgd_clf.fit(X_train_prepared, y_train['sunny'])

SGDClassifier(random_state=42)

In [36]:
flat_data = []
for i, direc in enumerate(X_val):
    img = imread(direc,as_gray=True)
    img_resized = resize(img, (60, 60), anti_aliasing=True, mode='reflect')
    flat_data.append(img_resized)

X_test_hog = hogify.transform(flat_data)
X_test_prepared = scalify.transform(X_test_hog)

ValueError: X has 1296 features, but this StandardScaler is expecting 4356 features as input.

In [23]:
y_pred = sgd_clf.predict(X_test_prepared)
print('Percentage correct: ', 100*np.sum(y_pred == y_val['sunny'])/len(y_val))

Percentage correct:  79.484375


In [4]:
flat_data = []
for i, direc in enumerate(X_train):
    img = imread(direc,as_gray=True)
#     img_resized = resize(img, (60, 60), anti_aliasing=True, mode='reflect')
    flat_data.append(img) 
X_train = np.array(flat_data)

# flat_data = []
# for i, direc in enumerate(X_val):
#     img = imread(direc,as_gray=True)
#     img_resized = resize(img, (60, 60), anti_aliasing=True, mode='reflect')
#     flat_data.append(img_resized.flatten()) 
# X_val = np.array(flat_data)

In [31]:
svc = LinearSVC(C=1.0)

# param_grid = [
#   {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
#  ]

# clf = GridSearchCV(svc, param_grid)
svc.fit(X_train_prepared, y_train['sunny'])

C:\Users\Gyu\AppData\Roaming\Python\Python36\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC()

In [32]:
svc.score(X_test_prepared, y_val['sunny'])

0.768359375

In [29]:
y_train['sunny'].value_counts()

0    31974
1    19226
Name: sunny, dtype: int64